In [29]:
import numpy as np 
import pandas as pd
import torch
from sklearn.metrics import r2_score
from pyro import clear_param_store
import pyro as pyro
import pyro.contrib.gp as gp
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive,HMC
import arviz as az
from scipy.optimize import curve_fit
from sklearn.metrics import root_mean_squared_error

In [30]:
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,examine,help"

In [31]:
PM25=pd.read_excel("../data/pm_flow.xlsx")
train,test=PM25.loc[PM25.label==0,:],PM25.loc[PM25.label==1,:]

In [32]:
train.shape,test.shape

((913, 24), (207, 24))

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [34]:
X_pm25=torch.tensor(train.flow.values).float().to(device)
y_pm25=torch.tensor(train.pm_final.values).float().to(device)
X_pm25_test=torch.tensor(test.flow.values).float().to(device)
y_pm25_test=test.pm_final.values

In [35]:
clear_param_store();
rbf = gp.kernels.RBF(input_dim=1);

rbf.variance = PyroSample(dist.HalfNormal(torch.tensor(80.)))

rbf.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(3.)))
gpr = gp.models.GPRegression(X_pm25,y_pm25, rbf).to(device);
gpr.noise = PyroSample(dist.HalfNormal(torch.tensor(100.)))

nuts_kernel = NUTS(gpr.model,target_accept_prob=0.95);

mcmc = MCMC(nuts_kernel, num_samples=3000,warmup_steps=9000,num_chains=1);

mcmc.run();

Sample: 100%|██████████████████████████████████████| 12000/12000 [13:25, 14.91it/s, step size=5.16e-01, acc. prob=0.961]


In [36]:
r2_score(y_pm25_test,gpr(X_pm25_test)[0].cpu().detach().numpy())

-0.4036811757508556

In [37]:
torch.save(gpr, "../models/PM2.5_flow");

In [38]:
mcmc.diagnostics()

{'kernel.lengthscale': OrderedDict([('n_eff',
               tensor(2513.2524, device='cuda:0')),
              ('r_hat', tensor(0.9998, device='cuda:0'))]),
 'kernel.variance': OrderedDict([('n_eff', tensor(2945.6431, device='cuda:0')),
              ('r_hat', tensor(0.9997, device='cuda:0'))]),
 'noise': OrderedDict([('n_eff', tensor(3190.1304, device='cuda:0')),
              ('r_hat', tensor(0.9998, device='cuda:0'))]),
 'divergences': {'chain 0': []},
 'acceptance rate': {'chain 0': 0.9986666666666667}}

In [39]:
posterior_samples = mcmc.get_samples();
posterior_predictive = Predictive(gpr, posterior_samples)(X_pm25);
prior = Predictive(gpr, num_samples=500)(X_pm25);

pyro_data = az.from_pyro(mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive

);
az.to_json(pyro_data, "../Arviz_stats/mcmc_pm25_flow.json")

/data/michaelf/miniconda3/lib/python3.10/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


'../Arviz_stats/mcmc_pm25_flow.json'

In [40]:
az.style.use("arviz-doc")

ax = az.plot_trace(pyro_data,backend='bokeh')

In [41]:
X_bc=torch.tensor(train.flow.values).float().to(device)
y_bc=torch.tensor(train.sv_final.values).float().to(device)
X_bc_test=torch.tensor(test.flow.values).float().to(device)
y_bc_test=test.sv_final.values

In [42]:
clear_param_store()
rbf_bc = gp.kernels.RBF(input_dim=1)

rbf_bc.variance = PyroSample(dist.HalfNormal(torch.tensor(80.)))

rbf_bc.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(3.)))
gpr_bc = gp.models.GPRegression(X_bc,y_bc, rbf_bc).to(device)
gpr_bc.noise = PyroSample(dist.HalfNormal(torch.tensor(2.)))


nuts_kernel_bc = NUTS(gpr_bc.model,target_accept_prob=0.95)


mcmc_bc = MCMC(nuts_kernel_bc,warmup_steps=9000, num_samples=3000,num_chains=1)

mcmc_bc.run()

Sample: 100%|██████████████████████████████████████| 12000/12000 [13:55, 14.36it/s, step size=2.69e-01, acc. prob=0.962]


In [43]:
r2_score(y_bc_test,gpr_bc(X_bc_test)[0].cpu().detach().numpy())

0.011349292974327851

In [44]:
root_mean_squared_error(y_bc_test,gpr_bc(X_bc_test)[0].cpu().detach().numpy())

0.5327479143108957

In [45]:
torch.save(gpr_bc, "../models/bc_flow");

In [46]:
posterior_samples_bc = mcmc_bc.get_samples(500)
posterior_predictive_bc = Predictive(gpr_bc, posterior_samples_bc)(X_bc)
prior_bc = Predictive(gpr_bc, num_samples=500)(X_bc)

pyro_data_bc = az.from_pyro(mcmc_bc,
    prior=prior_bc,
    posterior_predictive=posterior_predictive_bc,

)
az.to_json(pyro_data_bc, "../Arviz_stats/mcmc_bc_flow.json")

/data/michaelf/miniconda3/lib/python3.10/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


'../Arviz_stats/mcmc_bc_flow.json'